# ML Model use cases deployments - AI Pipeline in CP4D
_**Using AI pipelines to automate ML model deployment in Sagemaker and Watson OpenScale**_

### how to use <i>mlmonitor</i> client
- [Initialize Pipelines Client and variables](#pipelines_client)
- [Test cell to verify notebook execution](#test_cell)
- [Run all steps](#all_steps)
- [Train model in Sagemaker Job](#train_sm_job)
- [Deploy Online model](#deploy_sm_endpoint)
- [Score model](#score_endpoint)
- [Configure Monitoring](#monitor_endpoint)
- [Configure Quality](#quality_monitor)
- [Configure Explainability](#explain_monitor)
- [Configure Fairness Monitoring](#fairness_monitor)
- [Configure Drift Monitoring](#drift_monitor)
- [Log feedback data](#feedback_logging)
- [Log payload data](#payload_logging)
- [Evaluate monitors](#evaluate_monitors)
- [Report completion to MLOps orchestrator](#completed)
- [Delete Deployment](#delete_resources)
---

### Set up Project Token and API Key  <a name="tokens"></a>
Before executing this notebook, you need to insert a project token and create 2 project data assets :<br>


- `credentials_uploaded2cp4d.cfg` populated as described in [Readme](https://github.ibm.com/ds-client-engineering/model-monitoring-use-cases/blob/main/mlmonitor/README.md) section 2.1

- `mlmonitor-1.1-py3-none-any.whl` built as described in same Readme as above with `python setup_mlmonitor.py bdist_wheel`

#### Provide the project access token
1. When you imported the project from the github repo, the first cell of this notebook should contain the project access token. 

2. If this notebook does not contain a first cell with a project access token, then to generate the token, click **More > Insert project token** on the menu bar.<br>

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)


#### setup credentials for mlmonitor library

In [ ]:
import json
import os
import sys

cfg_filename='credentials_uploaded2cp4d.cfg'

if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D 
    mlmonitor_credentials = json.load(project.get_file(cfg_filename))


    with open("/tmp/credentials.cfg", "w") as f:
        json.dump(mlmonitor_credentials, f, indent=4, sort_keys=True)

    os.environ['MONITOR_CONFIG_FILE'] = "/tmp/credentials.cfg"

else:
     # RUN NOTEBOOK locally   

    with open(f"../{cfg_filename}", "r") as f:
        mlmonitor_credentials = json.load(f)

    os.environ['MONITOR_CONFIG_FILE'] = f"../{cfg_filename}"

    sys.path.append('../')

#### install <i>mlmonitor</i> from whl file (uploaded as data asset)

In [ ]:
if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D
    mlmonitor_whl = 'mlmonitor-1.1-py3-none-any.whl'
    filename = f"/tmp/{mlmonitor_whl}"

    obj = project.get_file(mlmonitor_whl)

    with open(filename, "wb") as out:
        out.write(obj.read())  

In [ ]:
if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D
    !pip install ibm_watson_studio_pipelines
    !pip -q uninstall mlmonitor -y

In [ ]:
if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D
    !pip -q install $filename
    !pip install numpy==1.23

### Test cell <a name="test_cell"></a>

In [ ]:
################################################################################################
# For testing purposes, uncomment model use case and task sections to be executed.             #
# To orchestrate this notebook in CP4D all lines of this cell should be commented out          #
# since environment variables are set at job creation                                          #
################################################################################################

#churn use evaluate
#catalog_id = '7630c102-42e6-40e6-a022-1ae98ba7ab8d'
# model_entry_id = '2000daab-cd35-4f00-82ce-dbb00d8a71a6'
# model_endpoint = 'sm-cc-xgboost-2023-01-26-02-58-07-897'
# source_dir = 'aws_code_churn'

#credit risk use case
#source_dir = 'aws_code_gcr'
#catalog_id = '7630c102-42e6-40e6-a022-1ae98ba7ab8d'
# model_entry_id = '58a50d85-1ff0-4c7e-b192-d8d9d33c4fe6'
# model_endpoint = 'wml-gcr-scikit-2023-02-15-09-00-00-008'


# tensorflow cnn MNIST dataset use case
# source_dir = 'aws_code_mnist_tf'
#catalog_id = '7630c102-42e6-40e6-a022-1ae98ba7ab8d'
# model_entry_id = '8881ef6b-1368-4f29-8f70-3c353a8b742c'
# model_endpoint = 'sm-mnist-cnn-tf-keras-2023-01-27-06-14-23-882'


## test all tasks - deploy - monitor - configure monitors - payload/feedback logging - evaluation
#os.environ['TASK']='all'
#os.environ['TASK']='train'
#os.environ['TASK']='deploy'
#os.environ['TASK']='monitor'
#os.environ['TASK']='configure_fairness'
#os.environ['TASK']='configure_drift'
#os.environ['TASK']='configure_quality'
#os.environ['TASK']='configure_explainability'
#os.environ['TASK']='configure_drift'
#os.environ['TASK']='payload_logging'
#os.environ['TASK']='feedback_logging'
#os.environ['TASK']='evaluate'
os.environ['TASK']='delete'

os.environ['source_dir']=source_dir
os.environ['MODEL_ENTRY_ID']=model_entry_id
os.environ['CATALOG_ID']=catalog_id
os.environ['deployment_target']='wml' # 'aws'
os.environ['job_name']=model_endpoint
os.environ['model_endpoint']=model_endpoint

#### this notebook should be executed for a specific task

In [ ]:
if not os.getenv('TASK'):
    raise ValueError('TASK should be set as environment variable')

In [ ]:
from mlmonitor.src.wml import wml_client,WML_SPACE_ID
wml_client.set.default_space(WML_SPACE_ID)

#### create decorator for validation steps

In [ ]:
from mlmonitor import SageMakerModelUseCase,WMLModelUseCase
from mlmonitor.src.utils.utils import complete_dag
from mlmonitor.src import API_KEY

def generic_task(function_task):
    
    def inner():

        if not all(k in os.environ for k in ("CATALOG_ID", "MODEL_ENTRY_ID", "model_endpoint", "deployment_target")):
            raise EnvironmentError('CATALOG_ID, MODEL_ENTRY_ID model_endpoint and deployment_target should be passed in ENV variables')
        
        if os.getenv('deployment_target') == 'aws':
            SelectedModelUseCase = SageMakerModelUseCase
        elif os.getenv('deployment_target') == 'wml':
            SelectedModelUseCase = WMLModelUseCase
        else:
            raise NotImplementedError("deployment_target should be aws or wml")

        model_use_case = SelectedModelUseCase(source_dir=os.getenv('source_dir','aws_code_churn'),
                               catalog_id=os.getenv('CATALOG_ID'),
                               model_entry_id=os.getenv('MODEL_ENTRY_ID'))

        if os.environ['TASK'] != 'train' :
            model_use_case.derive_model_states(endpoint_name=os.getenv('model_endpoint'))

        model_use_case.display_states()
        function_task(model_use_case)
        model_use_case.display_states()

        if os.getenv('deployment_target') == 'aws':
            os.environ['model_endpoint']=model_use_case.job_name

        if 'PROJECT_ID' in os.environ:
        # RUN NOTEBOOK in CP4D
            pipelines_client.store_results({'source_dir':source_dir,'catalog_id':catalog_id,'model_entry_id':model_entry_id})
    
    return inner


@generic_task
def train(uc):
    uc.train()
    
@generic_task
def deploy(uc):
    uc.deploy()
    
@generic_task
def monitor(uc):
    uc.monitor()
    
@generic_task
def govern(uc):
    uc.govern()
    

@generic_task
def log_payload(uc):
    uc.log_payload()
    

@generic_task
def log_payload(uc):
    uc.log_feedback()
    

@generic_task
def configure_quality_monitor(uc):
    uc.configure_quality_monitor()
    

@generic_task
def configure_fairness_monitor(uc):
    uc.configure_fairness_monitor()
    

@generic_task
def configure_explainability_monitor(uc):
    uc.configure_explainability_monitor()
    
@generic_task
def configure_drift_monitor(uc):
    uc.configure_drift_monitor()
    
    
@generic_task
def evaluate_model(uc):
    uc.evaluate_model()
    
    
@generic_task
def score_model(uc):
    uc.score_model()
    

@generic_task
def delete(uc):
    uc.cleanup()

### Initialize Pipelines Client and variables <a name="pipelines_client"></a>

In [ ]:
if 'PROJECT_ID' in os.environ:
    from ibm_watson_studio_pipelines import WSPipelines
    # RUN NOTEBOOK in CP4D
    pipelines_client = WSPipelines.from_apikey(API_KEY)
    

catalog_id = os.getenv('CATALOG_ID')
model_entry_id = os.getenv('MODEL_ENTRY_ID')
model_endpoint = os.getenv('model_endpoint')
source_dir = os.getenv('source_dir')
jobname = os.getenv('job_name')

### Run All Steps (assuming model is trained) <a name="all_steps"></a>

In [ ]:
if os.environ['TASK'] == 'all' :
    
    if not all(k in os.environ for k in ("CATALOG_ID", "MODEL_ENTRY_ID", "job_name","deployment_target", "source_dir")):
        raise EnvironmentError('job_name deployment_target and source_dir should be passed in ENV variables')
            
    if os.getenv('deployment_target') == 'aws':
            SelectedModelUseCase = SageMakerModelUseCase
    elif os.getenv('deployment_target') == 'wml':
            SelectedModelUseCase = WMLModelUseCase
    
    model_use_case = SelectedModelUseCase(source_dir=source_dir,catalog_id=catalog_id,model_entry_id=model_entry_id)
    model_use_case.derive_model_states(endpoint_name=jobname)
    model_use_case.display_states()
    
    if os.getenv('deployment_target') == 'wml':
        model_use_case.train()

    model_use_case.deploy()
    model_use_case.display_states()

    if os.getenv('deployment_target') == 'aws':
        model_use_case.govern()
        model_use_case.display_states()
    
    model_use_case.monitor()
    model_use_case.display_states()
    
    model_use_case.configure_quality_monitor()
    
    model_use_case.configure_fairness_monitor()
    
    model_use_case.configure_explainability_monitor()
    
    model_use_case.configure_drift_monitor()
    
    model_use_case.log_payload()
    model_use_case.log_feedback()
    
    model_use_case.evaluate_model()
    model_use_case.save_use_case(file=f"./{source_dir}_completed.json")

    model_endpoint=model_use_case.model_endpoint
    
    print(json.dumps(model_use_case.json_object(), indent=4))

### Train model in Sagemaker Job <a name="train_sm_job"></a>

In [ ]:
if os.environ['TASK'] == 'train':
    train()

### Delete Deployment <a name="delete_resources"></a>

In [ ]:
if os.environ['TASK'] == 'delete':
    delete()

### Deploy online endpoint <a name="deploy_sm_endpoint"></a>

In [ ]:
if os.environ['TASK'] == 'deploy' :
    deploy()

### Score model <a name="score_endpoint"></a>

In [ ]:
if os.environ['TASK'] == 'monitor':
    score_model()

### Configure Monitoring <a name="monitor_endpoint"></a>

In [ ]:
if os.environ['TASK'] == 'monitor':
    monitor()

In [ ]:
if os.environ['TASK'] == 'monitor':
    govern()

### Configure Quality Monitor <a name="quality_monitor"></a>

In [ ]:
if os.environ['TASK'] == 'configure_quality':
    configure_quality()

### Configure Explainability Monitor <a name="explain_monitor"></a>

In [ ]:
if os.environ['TASK'] == 'configure_explainability':
    configure_quality_monitor()

### Configure Fairness Monitor <a name="fairness_monitor"></a>

In [ ]:
if os.environ['TASK'] == 'configure_fairness':
    configure_fairness_monitor()

### Configure Drift Monitor <a name="drift_monitor"></a>

In [ ]:
if os.environ['TASK'] == 'configure_drift':
    configure_drift_monitor()

### Log feedback data <a name="feedback_logging"></a>

In [ ]:
if os.environ['TASK'] == 'feedback_logging' :
    log_payload()

### Log Payload data <a name="payload_logging"></a>

In [ ]:
if os.environ['TASK'] == 'payload_logging':
    log_payload()

### Evaluate monitors <a name="evaluate_monitors"></a>

In [ ]:
if os.environ['TASK'] == 'evaluate':
    evaluate_model()

### Report Change Request completion to MLOps Orchestrator Service <a name="completed"></a>

In [ ]:
if os.environ['TASK'] == 'test':
    
    model_endpoint = os.getenv('job_name')

In [ ]:
if os.environ['TASK'] in ['all','evaluate','delete', 'test'] :
        
    request_body = {

        "endpoint_name": model_endpoint,
        "status" : "Completed",
        "task" : os.environ['TASK']
    }

    username = mlmonitor_credentials.get('mlops_orchestrator').get('username')
    password = mlmonitor_credentials.get('mlops_orchestrator').get('password')
    scoring_url = mlmonitor_credentials.get('mlops_orchestrator').get('scoring_url')
    request_headers = {"Content-Type": "application/json"}
        
    print(request_body)
    
    resp = complete_dag(scoring_url=scoring_url,
                        username=username,
                        password=password,
                        request_headers=request_headers,
                        request_body=request_body)

    print(resp)